# 함수 (Function)
## 클로저 (Closure)
## @escaping 과 @autoclosure

- ### 클로저를 함수나 메소드의 인자값으로 사용할 때에는 용도에 따라 @escaping과 @autoclosure 속성을 부여할 수 있다

### @escaping
- #### @escaping 속성은 인자값으로 전달된 클로저를 저장해 두었다가, 나중에 다른 곳에서도 실행 할 수 있도록 허용해주는 속성이다

### 예제

```
func callback(fn: () -> Void) {
    fn()
}

callback {
    print("Closure가 실행되었습니다")
}
// "Closure가 실행되었습니다."
```

- 정의된 함수 callback(fn:)은 매개변수를 통해 전달된 클로저를 함수 내부에서 실행하는 역활을 한다

### 이번에는 이 코드를 다음과 같이 바꿔 보았다

```
func callback(fn: () -> Void) {
    let f = fn // 클로저를 상수 f에 대입
    f() // 대입된 클로저를 실행
}
```

- #### 바뀐 내용은 그리 많지 않다
    - #### 전달된 클로저를 우선 변수에 대입한 후에 실행하도록 과정을 추가했다

### 그런데 이를 실행하면 다음과 같은 오류가 출력된다

```
Non-escaping parameter'fn' may only be called
```
- #### 오류의 내용은 Non-escaping 파라미터인 'fn'은 오직 직접 호출하는 것만 가능하다는 의미이다
    - #### 이를 이해하기 위해서는 인자값으로 전달되는 클로저의 특성을 알아야 한다

### 스위프트에서 함수의 인자값으로 전달된 클로저는 기본적으로 "탈출불가(non-escape)"의 성격을 가진다
- #### 이는 해당 클로저를 "1.함수 내에서 2.직접 실행을 위해서만 사용"해야 하는 것을 의미
    - #### 이 때문에 함수 내부라 할지라도 변수나 상수에 대입할 수 없다
- #### 변수나 상수에 대입하는 것을 허용한다면 내부 함수를 통한 캡쳐(Capture)기능을 이용하여 클로저가 함수 바깥으로 탈출할 수 있기 때문이다
    - #### 여기서 말하는 탈출이란, 함수 내부 범위를 벗어나서 실행되는 것을 의미한다

### 동일한 의미에서, 인자값으로 전달된 클로저는 중첩된 내부 함수에서 사용할 수도 없다
- #### 내부 함수에서 사용할 수 있도록 허용할 경우, 이 역시 컨텍스트(Context)의 캡쳐를 통해 탈출될 수 있기 때문이다

### 따라서 다음의 예제를 실행하면 오류가 발생한다

```
func callback(fn: () -> Void) {
    func innerCallback() {
        fn()
    }
}
```

- ### 하지만 코드를 작성하다 보면 클로저를 변수나 상수에 대입하거나 중첩 함수 내부에서 사용해야 할 경우도 있다

### 이때 사용되는 것이 @escaping 속성이다
- #### 이 속성을 클로저에 붙여주면, 해당 클로저는 탈출이 가능한 인자값으로 설정이된다
- #### 앞서 살펴봤던 제약 조건들이 모두 제거되는 것이다.

### callback(fn:)함수의 매개변수 타입에 @escape 속성을 추가해 보자
- #### 이때 @escaping 속성이 붙는 위치에 주의해야 한다
- #### 이 속성은 인자값에 설정되는 값이므로, 함수 타입 앞에 넣어주어야 한다

```
func callback(fn: @escaping () -> Void) {
    let f = fn // 클로저를 상수 f에 대입
    f() // 대입된 클로저를 실행
}

callback {
    print("Closure가 실행되었습니다.")
}
// "Closure가 실행되었습니다."
```

- #### 이제 입력된 클로저는 변수나 상수에 정상적으로 할당될 뿐만 아니라, 중첩된 내부 함수에 사용 할 수 있으며, 함수 바깥으로 전달할 수도 있다
- #### 말 그대로 탈출 가능한 클로저가 된 것이다

### 그렇다면 인자값으로 전달되는 클로저의 기본 속성이 탈출불가하도록 설정된 이유는 무엇일까??
- #### 클로저의 기본 속성을 탈출불가(non-escape)하게 관리함으로써 얻어지는 가장 큰 이점은 컴파일러가 코드를 최적화하는 과정에서의 성능향상
- #### 해당 클로저가 탈출할 수 없다는 것은 컴파일러가 더 이상 메모리 관리상의 지저분한 일들에 관여할 필요없다는 뜻이기 때문이다

### 또한, 탈출불가 클로저 내에서는 self 키워드를 사용할 수 있다
- #### 왜냐하면 이 클로저는 해당 함수가 끝나서 리턴되기 전에 호출될 것이 명확하기 때문이다
- #### 따라서 클로저 내에서 self에 대한 약한 참조(weak reference)를 사용해야할 필요가 없다